# The analysis of multiscale model results & running TDSE interactively

This notebook shows various analyses of the results of the multiscale model. First, we investigate the pulse shaping due to the non-linear propagation, the plasma channel. Then we analyse the build-up of the XUV signal and its structure. Next, we will get more insight directly into the generating process inside the medium by studying the spectra. To conclude the analysis, we outline possibilities to fasten the analysis. Finally, we show the TDSE solver applied to a custom electric field defined directly in this notebook. We use various tools to analyse the result.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

We load the data from the pulse propagation in the Pythonic data container. It contains the data about the pulse propagation and some firther characteristics. The data from the micrscopic response and harmonic signal will be loaded later (these data are large, and we will need only a part of them according to the chosen analyses.)

In [ ]:
simulation = 'big'
h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', simulation, 'results.h5')
h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', simulation, 'results_Hankel.h5')

# simulation = 'bigpf'
# h5file1 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results.h5')
# h5file2 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results_Hankel.h5')

with h5py.File(h5file1,'r') as f:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f)
    CUPRAD_res.get_plasma(f)


## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:

tlim = np.asarray((-25,25))
rlim = 150

ani_outpath = os.path.join('/mnt','d', 'git', 'MMA-interactive', 'export')

In [ ]:
# Code to generate the animated figure

k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 1]})

r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
                    (
                    HHG.ComputeCutoff(
                        mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                        mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                        mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                    )[1]
                    ).T)

pc = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

ax1.set_xlim(tlim)
ax1.set_ylim((-rlim,rlim))

ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

cbar = fig.colorbar(pc, ax=ax1)
cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)


ax2.plot(1e3*CUPRAD_res.density_mod_zgrid,
            CUPRAD_res.density_mod_profile_mbar)
ax2.set_xlabel(r'$z~[\mathrm{mm}]$')
ax2.set_ylabel(r'$p~[\mathrm{mbar}]$')

progress_line, = ax2.plot([], [], 'r-')  

ax2.fill_between(1e3*CUPRAD_res.density_mod_zgrid, CUPRAD_res.density_mod_profile_mbar,color='lightgrey')
ax2.set_xlim((1e3*CUPRAD_res.density_mod_zgrid[0],1e3*CUPRAD_res.density_mod_zgrid[-1]))

def update(frame):
    # Update the data
    data = (mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max], (
            HHG.ComputeCutoff(
                        mn.FieldToIntensitySI(CUPRAD_res.E_zrt[frame,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                        mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                        mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                    )[1]        
            ).T)[1]).T
    
    # Update the colors
    pc.set_array(data.ravel())
    pc.set_clim(data.min(), data.max())
    cbar.update_normal(pc)

    ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame]) + ' mm')

    # Update the progress indicator
    progress_line.set_data([1e3*CUPRAD_res.zgrid[frame], 1e3*CUPRAD_res.zgrid[frame]],
                            [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

    return [pc, progress_line]

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid), blit=True)

HTML(ani.to_jshtml())

# Define the writer using ffmpeg for mp4 format and save it
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=30, metadata=dict(artist='Your Name'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'Gaussian_jet_pulse.mp4'), writer=writer)


In [ ]:
# Code to generate the figure of plasma channel


fig = plt.figure(figsize=(14, 5.5))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right


symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res.plasma.rgrid,CUPRAD_res.plasma.value_zrt[:,:,-1])
pc1 = ax1.pcolormesh(1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto')
ax1.set_ylim(-rlim,rlim)
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


local_density_modulation = CUPRAD_res.effective_neutral_particle_density *\
                                interpolate.interp1d(
                                CUPRAD_res.density_mod_zgrid,
                                CUPRAD_res.density_mod_profile_relative,
                                bounds_error = False,
                                fill_value = (CUPRAD_res.density_mod_profile_relative[0],
                                            CUPRAD_res.density_mod_profile_relative[-1]),
                                copy = False
                                )(CUPRAD_res.plasma.zgrid)

pc2 = ax2.pcolormesh(1e3*CUPRAD_res.plasma.zgrid,
               1e6*symmetric_y,
               1e2*(mn.symmetrize_y(CUPRAD_res.plasma.rgrid,
                        CUPRAD_res.plasma.value_zrt[:,:,-1] /
                            np.outer(np.ones(len(CUPRAD_res.plasma.rgrid)),
                                        local_density_modulation).T
                        )[1]).T,
                shading = 'auto')
ax2.set_ylim(-rlim,rlim)

cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')

ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
ax2.set_xlabel(r'$z~[\mathrm{mm}]$')

cbar1.ax.set_xlabel('plasma density $[\mathrm{m}^{-3}]$')
cbar2.ax.set_xlabel('relative plasma density [%]')

plt.show()


## XUV camera

Here we show the far-field XUV spectra together with the build-up of the signal in the generating medium.

These data might be very large. Please specify here the parametes of the plot, so only the necessary subset of data is loaded and processed.

In [ ]:
rmax = 0.007                # [m]    the radial dimesion to read the data
XUV_theta_range = [-4, 4]   # [mrad] the divergence angle for plotting 
orders_to_plot = 4          # the range of the logarithmic plot of the spatially resolved harmonic spectra
# Hmax_plot_linear = 50       # the maximal frequency in the linear plot of the spatially resolved harmonic spectra


kz_step = 10    #       the step in $z$ for plotting (derived from the spacing used in the computational grid)

H_interest = np.asarray([19, 25, 37, 49])   # harmonics for which we show the build-up
multipliers = [1,4,10,150]                  # multipliers applied to the build-up to fit the figure nicely
delta_H = 1.                                # the camera spectral range to analyse signal of the harmoonics of the interest

In [ ]:
# import data & basic analyses
with h5py.File(h5file1,'r') as f1,  h5py.File(h5file2,'r') as f2:
    # load Hankel data = XUV camera
    ogrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/zgrid'][0:-1:kz_step]

    camera_distance = mn.readscalardataset(f1,MMA.paths['Hankel_inputs']+'/distance_FF','N')
    theta_grid_Hankel = np.arctan(rgrid_Hankel/camera_distance)  # recompute the radial grid to the divergence

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0

    kr_max = mn.FindInterval(rgrid_Hankel, rmax) + 1
    rgrid_Hankel = rgrid_Hankel[:kr_max]
    theta_grid_Hankel = theta_grid_Hankel[:kr_max]


    cummulative_field =    f2[MMA.paths['Hankel_outputs']+'/cummulative_field'][0:-1:kz_step,:kr_max,:,0] +\
                    1j*f2[MMA.paths['Hankel_outputs']+'/cummulative_field'][0:-1:kz_step,:kr_max,:,1] 

# find maxima of the harmonics of the interest
H_idx = [tuple(mn.FindInterval(Hgrid_Hankel,(H_interest[k1]-delta_H, H_interest[k1]+delta_H))) for k1 in range(len(H_interest))]
H_max_interest = [np.max(np.abs(cummulative_field[:,:,H_idx[k1][0]:H_idx[k1][1]]),axis=(1,2)) for k1 in range(len(H_interest))]    

In [ ]:
# Code to create the following animated figure
fig = plt.figure(figsize=(14, 6))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)  # Upper left
ax2 = plt.subplot2grid((3, 2), (0, 1), rowspan=2)  # Upper right
ax3 = plt.subplot2grid((3, 2), (2, 0), colspan=2)  # Bottom, spanning both columns

r_grid, sym_data = mn.symmetrize_y(rgrid_Hankel, np.abs(cummulative_field[0,:,:]).T)
theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cummulative_field[0,:,:]).T)



pc1 = ax1.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
pc2 = ax2.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))

ax1.set_ylim(XUV_theta_range)
ax2.set_ylim(XUV_theta_range)

# ax1.set_xlim(Hgrid_Hankel[0],Hmax_plot_linear)

ax1.set_title('spatially resolved XUV spectrum (linscale)')
ax2.set_title('spatially resolved XUV spectrum (logscale)')

ax1.set_xlabel('harmonic order [-]')
ax2.set_xlabel('harmonic order [-]')

ax1.set_ylabel(r'divergence [mrad]')

cbar1 = fig.colorbar(pc1, ax=ax1)
cbar2 = fig.colorbar(pc2, ax=ax2) #, orientation='horizontal')
cbar2.ax.set_ylabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]', rotation=90)

# plot lines at selected harmonic orders
for k1 in range(len(H_interest)):
    ax1.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)
    ax2.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)



title = fig.suptitle("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')


ax3.plot(1e3*CUPRAD_res.density_mod_zgrid,
            CUPRAD_res.density_mod_profile_mbar,'gray')
ax3.set_xlabel(r'$z~[\mathrm{mm}]$')
ax3.set_ylabel(r'$p~[\mathrm{mbar}]$')

progress_line, = ax3.plot([], [], 'r-')  

ax3.fill_between(1e3*CUPRAD_res.density_mod_zgrid, CUPRAD_res.density_mod_profile_mbar,color='lightgrey')
ax3.set_xlim((1e3*CUPRAD_res.density_mod_zgrid[0],1e3*CUPRAD_res.density_mod_zgrid[-1]))

ax3_tw = ax3.twinx()

# normalised signals
max_signal = np.max([signal for signal in H_max_interest])
# multipliers = [max_signal/np.max(signal) for signal in H_max_interest]

for k1 in range(len(H_interest)):
    if (len(zgrid_Hankel) == len(H_max_interest[k1][:])): signal_plot = H_max_interest[k1][:]
    else: signal_plot = np.append(0,H_max_interest[k1][:])
    ax3_tw.plot(1e3*zgrid_Hankel,multipliers[k1]*signal_plot, label='H'+str(H_interest[k1])+f' (x {multipliers[k1]:.1f})')

ax3_tw.set_ylabel(r'XUV signal $[\mathrm{arb. u.}]$')

# ax3_tw.plot(1e3*zgrid_Hankel,np.append(0,H_max_interest[0][:]), 'b--', label='xxx')

ax3_tw.legend()

def update(frame):
    # Update the data
    data = (mn.symmetrize_y(rgrid_Hankel, np.abs(cummulative_field[frame,:,:]).T)[1]).T

    pc1.set_array(data.ravel())
    pc1.set_clim(data.min(), data.max())

    # Update the colors
    pc2.set_array(data.ravel())
    # pc.set_clim(data.min(), data.max())
    pc2.set_clim((10**(-orders_to_plot))*data.max(), data.max())
    # cbar.update_normal(pc2)

    title.set_text("z={:.2f}".format(1e3*zgrid_Hankel[frame+1]) + ' mm')

    # Update the progress indicator
    progress_line.set_data([1e3*zgrid_Hankel[frame+1], 1e3*zgrid_Hankel[frame+1]],
                            [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

    return [pc1,pc2, progress_line]


# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout()

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(zgrid_Hankel)-1, blit=True)
# ani = matplotlib.animation.FuncAnimation(fig, update, frames=3, blit=True)
HTML(ani.to_jshtml())

# Define the writer using ffmpeg for mp4 format and save it
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=3, metadata=dict(artist='Your Name'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'Gaussian_jet_spectra.mp4'), writer=writer)


## Microscopic insights

Here we investigate in more detail the generating process inside the medium. We take a single plane and look at the electric fields and microscopic source terms in the medium.

In [ ]:
z_analyse = (4./5.)*CUPRAD_res.zgrid[-1] # 0.5* # the z-coordinate to cut the plane
Hmax_plot = 50                                  # maximal harmonic shown in the plot

In [ ]:
# Code to create the following figure


kz_analyse = mn.FindInterval(CUPRAD_res.zgrid ,z_analyse)



with h5py.File(h5file1,'r') as f1:

    tlim = np.asarray((-25,25))
    rlim = 150
    k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
    k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

    # load TDSE data
    rgrid_TDSE = f1[MMA.paths['CTDSE_outputs'] +'/rgrid_coarse'][:]; Nr_TDSE = len(rgrid_TDSE)
    zgrid_TDSE = f1[MMA.paths['CTDSE_outputs'] +'/zgrid_coarse'][:]; Nz_TDSE = len(zgrid_TDSE)
    ogrid_TDSE = f1[MMA.paths['CTDSE_outputs'] +'/omegagrid'][:]
    tgrid_TDSE = f1[MMA.paths['CTDSE_outputs'] +'/tgrid'][:]
    Hgrid_TDSE = ogrid_TDSE/mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau')



    kz_analyse_TDSE = mn.FindInterval(zgrid_TDSE ,z_analyse)

    k_r_max_TDSE    = mn.FindInterval(1e6*rgrid_TDSE ,1.05*rlim)

    # it seems that h5py cannot easily provide data for the animation
    spectra_to_plot = [np.abs(    f1[MMA.paths['CTDSE_outputs'] +'/FSourceTerm'][kz_analyse_TDSE,k1,:,0] +
                              1j*f1[MMA.paths['CTDSE_outputs'] +'/FSourceTerm'][kz_analyse_TDSE,k1,:,1])
                       for k1 in range(Nr_TDSE)]

    Efields_to_plot = [f1[MMA.paths['CTDSE_outputs'] +'/Efield'][kz_analyse_TDSE,k1,:] for k1 in range(Nr_TDSE)]



    

    r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
                       (
                        HHG.ComputeCutoff(
                            mn.FieldToIntensitySI(CUPRAD_res.E_zrt[kz_analyse,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                            mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                            mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                        )[1]
                       ).T)

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # One row, three columns

    pc = axs[0].pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')
    cbar = fig.colorbar(pc, ax=axs[0], orientation = 'horizontal')
    progress_line, = axs[0].plot([], [], 'r-')  # Horizontally progressing line
    axs[0].set_xlabel(r'$t~[\mathrm{fs}]$')
    axs[0].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')
    cbar.ax.set_xlabel('Intensity [harmonic cut-off]')


    plot1, = axs[1].plot(1e15*(tgrid_TDSE-0.5*tgrid_TDSE[-1])*units.TIMEau, Efields_to_plot[0])
    axs[1].set_xlabel(r'$t~[\mathrm{fs}]$')
    axs[1].set_ylabel(r'$\mathcal{E}~[\mathrm{a.u.}]$')


    plot2, = axs[2].semilogy(Hgrid_TDSE, spectra_to_plot[0])
    axs[2].set_xlabel('harmonic order [-]')
    axs[2].set_ylabel(r'$\mathcal{E}_{\text{XUV}}~[\mathrm{arb.~u.}]$')

    axs[2].set_xlim((Hgrid_TDSE[0], Hmax_plot))

    def update(frame):
        # Update the progressing line in the pcolormesh plot
        progress_line.set_data([1e15*CUPRAD_res.tgrid[k_t_min],1e15*CUPRAD_res.tgrid[k_t_max]],
                                2*[1e6*rgrid_TDSE[frame]])
        

        plot1.set_ydata(Efields_to_plot[frame])
        plot2.set_ydata(spectra_to_plot[frame])
        
        return [progress_line, plot1, plot2] # , line1, line2]




    # Ensure the layout does not have overlaps and everything is nicely spaced
    fig.tight_layout()

    ani = matplotlib.animation.FuncAnimation(fig, update, frames=k_r_max_TDSE, blit=True)
    # ani = matplotlib.animation.FuncAnimation(fig, update, frames=3, blit=True)

HTML(ani.to_jshtml())

# Define the writer using ffmpeg for mp4 format and save it
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=20, metadata=dict(artist='Your Name'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'Gaussian_jet_micro_insight.mp4'), writer=writer)

### Another example: coherence map

Finally, we show an example from [Phase-matched high-order harmonic generation in pre-ionized noble gases](https://doi.org/10.1038/s41598-022-11313-6), where we analysed directly the outputs of CUPRAD to obtain "coherence maps". This provides a fast microscopic insight in the generation without the neeed of computing the costly microscopic response.

This analysis comes from the usual formula to evaluate phase mismatch characterised by
$$ \Delta k_q = \Delta k_{\text{neutral}} + \Delta k_{\text{plasma}} + \Delta k_{\text{geometry}} + \Delta k_{\text{atomic}} \,.$$
The phase matching is then characterised by the *coherence length* $L_{\text{coh}} = |\pi/\Delta k_q|$, which gives the spatial range where the microscopic emitters sum up consructively. In the case of a numerical field, we cannot separate different contrubutions ($\Delta k_{\text{neutral}}, \Delta k_{\text{plasma}}, \Delta k_{\text{geometry}}$). These quaontities are inherently present in the results, and can be obtained by differentiating the field's phase $\Delta k_{\text{num.}}=\partial \Phi_{\text{IR}} / \partial z$. 

The following [figures](https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-022-11313-6/MediaObjects/41598_2022_11313_MOESM1_ESM.pdf) shows the coherence maps for the generation of the 17<sup>th</sup> harmonic in a 15-mm-long gas cell filled by krypton at 36 mbar. 

![fig1](Lcoh_rz_I0_5_p13_eta0_H17.png) ![fig2](Lcoh_rz_I0_5_p13_eta1_H17.png)

The right figure shows the conditions except the gas is pre-ionised to 8 %, which significatly increases $L_{\text{coh}}$ and, therefore, the phase-matching. See the [reference](https://doi.org/10.1038/s41598-022-11313-6) for the details about the scheme and its experimental realisation.

## TDSE with a custom input

Now, we move to the microscopic response. We show the interface for the TDSE solver accessed directly through Python. We use this solver for a custom field we define, and then analyse the result in details. We will show the spectrum of the source term, wavefunction, we do energetic analyses via the Gabor transform and [invariant energetic distribution](https://doi.org/10.1103/PhysRevA.106.053115). FInally, we will show also the depletion of the ground state.


First, we import the compiled dynamical library and its Pythonic wrapper:

In [ ]:

# Main Python TDSE helper script, contains C structures, function wrappers etc.
sys.path.append('/mnt/d/git/CUPRAD_TDSE_Hankel/1DTDSE')
from PythonTDSE import *

# Compiled dynamic C library
path_to_DLL = '/mnt/d/git/CUPRAD_TDSE_Hankel/1DTDSE/build/libsingleTDSE.so'
DLL = TDSE_DLL(path_to_DLL)

### Define the custom input field & numerical parameters

Here we define the input parameters for the CTDSE solver and the initial pulse. We show an example of a chirped pulse with a $\sin^2$-envelope. The field is then given by

$$ \mathcal{E}(t) = \mathcal{E}_0 \sin^2 \left( \frac{t}{T_{\text{envelope}}} \right) \cos \left(\omega_0 t + \omega_c t^2 \right) \,.$$

(Note that the instantaneous frequency is then $\omega_i(t) = \omega_0 + 2\omega_c t$. This means that $\omega_0$ cannot be generally taken as the central frequency.)

In [ ]:
omega0 = mn.ConvertPhoton(1000e-9,'lambdaSI','omegaau')
chirp = 2e-4
E_0 = 0.15   # peak electric field amplitude

T0 = mn.ConvertPhoton(omega0,'omegaau','T0au') # the duration of the reference cycle
T_max = 3*T0 # total pulse duration expressed in the number of the reference cycles
N_t = 10000  # # of points for field construction (not for TDSE)

# Construct the field
tgrid = np.linspace(0, T_max, N_t)
E = E_0* (np.sin(np.pi*tgrid/T_max)**2) *np.cos(omega0*tgrid + chirp*(tgrid)**2)


# Create instance of input structure
inputs = inputs_def()

# Set the inputs for the TDSE solver
trg_a = 1.1893 # Argon 
inputs.init_default_inputs(
            Eguess   = -0.5145 ,
            trg_a    = trg_a ,     
            dt       = 0.125 ,
            dx       = 0.4 ,
            num_r    = 16000 ,
            writewft = 1 ,
            tprint   = 1. ,
            x_int    = 2. )
# Note: Parameters currently needs to be fixed for the gauge-invariant energetic analysis (gas & some of numerics for the same ensemble of bound states)

### Pipeline to execute the TDSE computation

In [ ]:
inputs.init_time_and_field(DLL, E = E, t = tgrid) # set our electric field as the input
DLL.init_GS(inputs)                               # create the C-types input for the C-library
output = outputs_def()                            # prepare the structure that holds the TDSE outputs 
DLL.call1DTDSE(inputs, output)                    # run TDSE

### Obtain detailed analyses and visualisation
Here we specify some parameters for various analyses and plotting

In [ ]:
# Parameters for analyses

# Gabor
omega_max_plot = 3.5 # [a.u.]
Tmin_Gabor = 20.     # [a.u.]
Tmax_Gabor = 380.    # [a.u.]


# Energetic distribution
E_min = -0.6    # [a.u.] - minimal energy in the analysis
E_max = 5.      # [a.u.] - maximal energy in the analysis
N_pts = 1500    # # of points in energy

# Numerical parameters
Nthreads = 10   # # of threads for the parallel computation

### Compute Gabor transform

The Gabor transform is one of the analyses provided directly by the C-library. 

In [ ]:
grad_V = output.get_sourceterm()
dt_Gabor = output.tgrid[1]-output.tgrid[0]
T = output.tgrid[output.Nt-1]

tgrid_Gabor, ogrid_Gabor, Gabor = DLL.gabor_transform(grad_V, dt_Gabor, output.Nt, omega_max_plot, Tmin_Gabor, Tmax_Gabor, 1000, a=8)

### Compute invariant energy distribution & ionisation probability (using `multiprocessing` parallelisation)

The computation of the invariant energy distribution is a computationally heavy task. It basically requires [to compute the photo-electron spectrum for each $t$](https://doi.org/10.1103/PhysRevA.106.053115https://doi.org/10.1103/PhysRevA.106.053115).

We use the `multiprocessing` module to compute the result in parallel. We need also some preparational computations: to find the ensemble of the bound states that will be projected out from the distribution. Then we define the computational routine to be parallelised by the `starmap_async`.

In [ ]:
# compute bound states that will be projected out
Energy_guess = [-0.5789, -0.2537, -0.1425, -0.0890, -0.0613, -0.0440, -0.0335, -0.0265, -0.0213, -0.0175, -0.0145, -0.0105, -0.0080, -0.0065, -0.0050, -0.0035]
inputs_array = []
GS = []
for i, E in enumerate(Energy_guess):
    inputs_array.append(inputs_def())
    inputs_array[i].init_default_inputs(Eguess=E, num_r=inputs.num_r, trg_a=trg_a, dt = 0.25, CV = 1e-15) # CV = 1e-15, else the resolvent does not converge for higher bound states
    DLL.init_GS(inputs_array[i]) 
    print("E_GS = {}".format(inputs_array[i].Einit))
    GS.append(inputs_array[i].get_GS())

GS = np.array(GS)

# Free the memory allocated by the temporary arrays for the energy computation
for i, E, in enumerate(inputs_array):
    inputs_array[i].delete(DLL)

In [ ]:
# define the function used for the parallelised computaiton of the photoelectron spectrum


t_psi, x_grid, wavefunction = output.get_wavefunction(inputs, grids=True)
wfs = wavefunction[0:-1:1]


def compute_PES_parallel(psi, GS, N_pts, E_min, E_max, jobID):
    for psi_b in GS:
        psi -= np.vdot(psi, psi_b)*psi_b # Remove the bound states using projection (note: np.vdot(a, b) == np.dot(np.conj(a), b) )

    ### Convert psi into a C-comprehensible array - flatten first, alternate real 
    # and imaginary parts, then return pointer to array
    #psi = np.array([psi.real, psi.imag]).transpose().flatten()
    #psi_ptr = ctypes_arr_ptr(c_double, len(psi), psi)

    ### Compute photoelectron spectrum for range [E_min, E_max]
    Energy = np.linspace(E_min, E_max, N_pts)
    Estep = Energy[1]-Energy[0]
    E_grid, PES = DLL.compute_PES(inputs, psi, num_E=len(Energy), Estep=Estep)

    # print("Job {} done.".format(jobID))
    print(f'Job {jobID} done.    \r', end="")
    
    ### Store the result
    return E_grid, PES

In [ ]:
# compute the analysis in parallel
from multiprocess import Pool
p = Pool(Nthreads)

map_ = [(wf, GS, N_pts, E_min, E_max, i+1) for i, wf in enumerate(wavefunction)]
result = p.starmap_async(compute_PES_parallel, map_)    # run calculation in parallel

PES = result.get()
PES_array = [PES[i][1] for i in range(len(PES))]        # collect the results

### Final plot of all data

In [ ]:
# code to generate the following figure
import matplotlib.colors as colors
fig = plt.figure(figsize=(14, 10))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((3, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((3, 2), (0, 1))  # Upper right
ax3 = plt.subplot2grid((3, 2), (1, 0))  # Middle left
ax4 = plt.subplot2grid((3, 2), (1, 1))  # Middle right
ax5 = plt.subplot2grid((3, 2), (2, 0))  # Lower left
ax6 = plt.subplot2grid((3, 2), (2, 1))  # Lower right


ax1.plot(output.get_tgrid(),output.get_Efield(),label='Electric field')
ax1.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax1.set_ylabel(r'$\mathcal{E}~[\mathrm{a.u.}]$')
# ax1.set_title('Eelctric field')
ax1.legend()


ogrid = output.get_omegagrid()[:]
ko_max = mn.FindInterval(ogrid,omega_max_plot)
ax2.semilogy(mn.ConvertPhoton(ogrid[:ko_max],'omegaau','eV'), np.abs(output.get_Fsourceterm())[:ko_max],label='dipole acceleration spectrum')
ax2.set_xlim(mn.ConvertPhoton(ogrid[:ko_max],'omegaau','eV')[[0,-1]])
ax2.set_xlabel(r'$\omega~[\mathrm{eV}]$')
ax2.set_ylabel(r'$|(\partial \hat{\jmath}/\partial t)(\omega)|~[\mathrm{arb.~u.}]$')
ax2.legend()
# plt.xlim(0,3.5)


x_range = (np.abs(x_grid) < 250.1)
pc3 = ax3.pcolormesh(t_psi, x_grid[x_range], np.transpose(np.abs(wavefunction))[x_range],
                     cmap = 'jet',
                     norm = colors.LogNorm(vmin=1e-8, vmax=0.5),
                     shading = 'gouraud')
ax3.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax3.set_ylabel(r'$x~[\mathrm{a.u.}]$')
cbar2 = fig.colorbar(pc3, ax=ax3) #, orientation='horizontal')
cbar2.ax.set_ylabel(r'$|\psi|$ [a.u.]', rotation=90)



pc4 = ax4.pcolormesh(tgrid_Gabor, mn.ConvertPhoton(ogrid_Gabor,'omegaau','eV'), Gabor,
                     cmap = 'jet',
                     # vmin = 1e-6,
                     norm = colors.LogNorm(vmin=1e-6, vmax=1), # Normalize
                     shading = 'gouraud')
ax4.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax4.set_ylabel(r'Energy [eV]')
cbar4 = fig.colorbar(pc4, ax=ax4) #, orientation='horizontal')
cbar4.ax.set_ylabel(r'Gabor (spectrogram) [arb. u.]', rotation=90)


# plot_colormap(ogrid_Gabor, tgrid_Gabor, Gabor, z_min=1e-6, figsize=(5, 3))

pc5 = ax5.pcolormesh(t_psi,
                     mn.ConvertPhoton(np.linspace(E_min, E_max, N_pts),'omegaau','eV'),
                     1e4*np.array(PES_array).transpose(),
                     cmap = 'jet', #'bwr',
                     # vmin = 1e-6,
                     norm = colors.Normalize(vmin=1e4*1e-8, vmax=1e4*1e-4), # Normalize
                     shading = 'gouraud')

ax5.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax5.set_ylabel(r'Energy [eV]')
cbar5 = fig.colorbar(pc5, ax=ax5) #, orientation='horizontal')
cbar5.ax.set_ylabel("ivnariant electron's energy [arb. u.]", rotation=90)


ax6.plot(output.get_tgrid(),output.get_PopInt(),label=r'Volumetric GS population ($\int_{V_{\text{atom}}} |\psi(t)|^2$)')
ax6.plot(output.get_tgrid(),output.get_PopTot(),label=r'Invariant projected GS pop. ($|\langle\psi(t)|\psi_0 \rangle_{\text{inv}}|^2$)')
ax6.set_xlim(output.get_tgrid()[[0,-1]])
ax6.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax6.set_ylabel(r'Probability [-]')
ax6.legend()


fig.tight_layout()
plt.show()